In [268]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import re
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore') 

sns.set_theme(style = "white", context = "talk",font_scale = 0.5)
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)

In [269]:
sc = pd.read_json('scout_car.json', lines=True)

In [270]:
sc_1 = sc.copy()

In [271]:
sc.shape

(15919, 54)

In [247]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,\nComfort & Convenience\n,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...","[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,NaN,85 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]",NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",[\nAM-95365\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nmythosschwarz metallic\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n4\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,598 cc\n]",NaN,NaN,[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.4 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Cruise control, Electrical ...","[MP3, On-board computer]","[Alloy wheels, Voice Control]","[ABS, Central door lock, Daytime running light...","[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",[\n4 (Green)\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]","[\n, \n, \nEuro 6\n]",[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2016, \n]","[\n, Brown, \n]",[\nMetallic\n],NaN,NaN,"[\n, Sedans, \n]",[\n3\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,195 kg\n]",NaN,"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[

In [272]:
sc.columns = sc.columns.str.strip("\n")
sc.columns = sc.columns.str.replace(' ', '_')
sc.columns = sc.columns.str.replace('-', '_')

In [273]:
sc = sc.drop(columns = ["kW", "null", "prev_owner", "Make", "Model", "Offer_Number", "First_Registration", "Electricity_consumption", "Last_Service_Date", "Other_Fuel_Types", "Availability", "Last_Timing_Belt_Service_Date", "Available_from"] )

In [274]:
cl_parantez = [ 'Type', 'Previous_Owners','Next_Inspection','Inspection_new','Warranty', 'Full_Service','Non_smoking_Vehicle', 'Body_Color', 'Paint_Type', 'Body_Color_Original', 'Upholstery', 'Body', 'Nr._of_Doors', 'Nr._of_Seats', 'Model_Code' ,'Gearing_Type', 'Displacement', 'Cylinders', 'Weight', 'Drive_chain', 'Fuel', 'Consumption', 'CO2_Emission', 'Emission_Class', 'Comfort_&_Convenience', 'Entertainment_&_Media' ,'Extras','Safety_&_Security','description','Emission_Label','Gears' ,'Country_version']

In [275]:
cl_parantez

['Type',
 'Previous_Owners',
 'Next_Inspection',
 'Inspection_new',
 'Warranty',
 'Full_Service',
 'Non_smoking_Vehicle',
 'Body_Color',
 'Paint_Type',
 'Body_Color_Original',
 'Upholstery',
 'Body',
 'Nr._of_Doors',
 'Nr._of_Seats',
 'Model_Code',
 'Gearing_Type',
 'Displacement',
 'Cylinders',
 'Weight',
 'Drive_chain',
 'Fuel',
 'Consumption',
 'CO2_Emission',
 'Emission_Class',
 'Comfort_&_Convenience',
 'Entertainment_&_Media',
 'Extras',
 'Safety_&_Security',
 'description',
 'Emission_Label',
 'Gears',
 'Country_version']

In [276]:
def try_join(l):
    try:
        return ''.join(map(str, l))
    except TypeError:
        return np.nan

In [277]:
for i in cl_parantez:
     sc[i] = [try_join(l) for l in sc[i]]

In [264]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,56013,01/2016,66,UsedDiesel (Particulate Filter),2,06/202199 g CO2/km (comb),YesEuro 6,4 (Green),NaN,NaN,Black,Metallic,Mythosschwarz,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,1422,3,1220,front,Diesel (Particulate Filter),381004310035100,992,Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsCatalytic ConverterVoice Control,ABSCentral door lockDaytime running lightsDriv...,Sicherheit: Deaktivierung für Beifahrer-Airbag...,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,80000,03/2017,141,UsedGasoline,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,"Cloth, Grey",Sedans,3,4,0588/BCY,Automatic,1798,4,1255,front,Gasoline,561007110047100,1292,Euro 6,Air conditioningAutomatic climate controlHill ...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsSport seatsSport suspensionVoice C...,ABSCentral door lockCentral door lock with rem...,Langstreckenfahrzeug daher die hohe Kilometerl...,4,7,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,83450,02/2016,85,UsedDiesel (Particulate Filter),1,NaN,NaN,99 g CO2/km (comb),NaN,NaN,Black,Metallic,mythosschwarz metallic,"Cloth, Black",Sedans,4,4,NaN,Automatic,1598,NaN,NaN,front,Diesel (Particulate Filter),381004410034100,992,Euro 6,Air conditioningCruise controlElectrical side ...,MP3On-board computer,Alloy wheelsVoice Control,ABSCentral door lockDaytime running lightsDriv...,Fahrzeug-Nummer: AM-95365 Ehem. UPE 24.640 EUR...,4,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,NaN,73000,08/2016,66,UsedDiesel (Particulate Filter),1,NaN,NaN,NaN,99 g CO2/km (comb),Euro 6,Brown,Metallic,NaN,NaN,Sedans,3,4,NaN,Automatic,1422,3,1195,NaN,Diesel (Particulate Filter),381004310035100,992,Euro 6,Air suspensionArmrestAuxiliary heatingElectric...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport seatsVoice Control,ABSAlarm systemCentral door lock with remote c...,Audi A1: - 1e eigenaar - Perfecte staat: schad...,NaN,6,NaN
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,NaN,16200,05/2016,66,UsedDiesel (Particulate Filter),1,NaN,Yes109 g CO2/km (comb),Euro 6,4 (Green),NaN,Black,Metallic,Mythosschwarz Metallic,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,1422,3,NaN,front,Diesel (Particulate Filter),411004610038100,1092,Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport packageSport suspensionVoice...,ABSCentral door lockDriver-side airbagElectron...,Technik & Sicherheit:Xenon plusKlimaautomatikE...,NaN,NaN,Germany


In [278]:
for i in cl_rem:
    sc[i] = sc[i].apply(lambda x: str(x).replace('\n', ''))

In [263]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,56013,01/2016,66,UsedDiesel (Particulate Filter),2,06/202199 g CO2/km (comb),YesEuro 6,4 (Green),NaN,NaN,Black,Metallic,Mythosschwarz,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,1422,3,1220,front,Diesel (Particulate Filter),381004310035100,992,Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsCatalytic ConverterVoice Control,ABSCentral door lockDaytime running lightsDriv...,Sicherheit: Deaktivierung für Beifahrer-Airbag...,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,80000,03/2017,141,UsedGasoline,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,"Cloth, Grey",Sedans,3,4,0588/BCY,Automatic,1798,4,1255,front,Gasoline,561007110047100,1292,Euro 6,Air conditioningAutomatic climate controlHill ...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsSport seatsSport suspensionVoice C...,ABSCentral door lockCentral door lock with rem...,Langstreckenfahrzeug daher die hohe Kilometerl...,4,7,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,83450,02/2016,85,UsedDiesel (Particulate Filter),1,NaN,NaN,99 g CO2/km (comb),NaN,NaN,Black,Metallic,mythosschwarz metallic,"Cloth, Black",Sedans,4,4,NaN,Automatic,1598,NaN,NaN,front,Diesel (Particulate Filter),381004410034100,992,Euro 6,Air conditioningCruise controlElectrical side ...,MP3On-board computer,Alloy wheelsVoice Control,ABSCentral door lockDaytime running lightsDriv...,Fahrzeug-Nummer: AM-95365 Ehem. UPE 24.640 EUR...,4,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,NaN,73000,08/2016,66,UsedDiesel (Particulate Filter),1,NaN,NaN,NaN,99 g CO2/km (comb),Euro 6,Brown,Metallic,NaN,NaN,Sedans,3,4,NaN,Automatic,1422,3,1195,NaN,Diesel (Particulate Filter),381004310035100,992,Euro 6,Air suspensionArmrestAuxiliary heatingElectric...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport seatsVoice Control,ABSAlarm systemCentral door lock with remote c...,Audi A1: - 1e eigenaar - Perfecte staat: schad...,NaN,6,NaN
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,NaN,16200,05/2016,66,UsedDiesel (Particulate Filter),1,NaN,Yes109 g CO2/km (comb),Euro 6,4 (Green),NaN,Black,Metallic,Mythosschwarz Metallic,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,1422,3,NaN,front,Diesel (Particulate Filter),411004610038100,1092,Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport packageSport suspensionVoice...,ABSCentral door lockDriver-side airbagElectron...,Technik & Sicherheit:Xenon plusKlimaautomatikE...,NaN,NaN,Germany


In [279]:
cl = list(sc.columns)
cl

['url',
 'make_model',
 'short_description',
 'body_type',
 'price',
 'vat',
 'km',
 'registration',
 'hp',
 'Type',
 'Previous_Owners',
 'Next_Inspection',
 'Inspection_new',
 'Warranty',
 'Full_Service',
 'Non_smoking_Vehicle',
 'Body_Color',
 'Paint_Type',
 'Body_Color_Original',
 'Upholstery',
 'Body',
 'Nr._of_Doors',
 'Nr._of_Seats',
 'Model_Code',
 'Gearing_Type',
 'Displacement',
 'Cylinders',
 'Weight',
 'Drive_chain',
 'Fuel',
 'Consumption',
 'CO2_Emission',
 'Emission_Class',
 'Comfort_&_Convenience',
 'Entertainment_&_Media',
 'Extras',
 'Safety_&_Security',
 'description',
 'Emission_Label',
 'Gears',
 'Country_version']

In [280]:
cl_rem = ['Previous_Owners', 'Next_Inspection','Inspection_new','Warranty', 'Full_Service', 'Non_smoking_Vehicle','Body_Color', 'Paint_Type','Body_Color_Original','Upholstery',
          'Body','Nr._of_Doors', 'Nr._of_Seats', 'Nr._of_Seats', 'Gearing_Type',
 'Displacement', 'description', 'Model_Code','Cylinders', 'Weight','Drive_chain','Fuel', 'CO2_Emission',  'Emission_Class' ,'Emission_Label' , 'Gears',
 'Country_version']

In [78]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,66 kW,UsedDiesel (Particulate Filter),2,06/202199 g CO2/km (comb),YesEuro 6,4 (Green),,,Black,Metallic,Mythosschwarz,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,"1,422 cc",3,"1,220 kg",front,Diesel (Particulate Filter),['3.8 l/100 km (comb)']['4.3 l/100 km (city)']...,99 g CO2/km (comb),Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsCatalytic ConverterVoice Control,ABSCentral door lockDaytime running lightsDriv...,Sicherheit: Deaktivierung für Beifahrer-Airbag...,nan,nan,nan
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,141 kW,UsedGasoline,nan,nan,nan,nan,nan,nan,Red,nan,nan,"Cloth, Grey",Sedans,3,4,0588/BCY,Automatic,"1,798 cc",4,"1,255 kg",front,Gasoline,['5.6 l/100 km (comb)']['7.1 l/100 km (city)']...,129 g CO2/km (comb),Euro 6,Air conditioningAutomatic climate controlHill ...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsSport seatsSport suspensionVoice C...,ABSCentral door lockCentral door lock with rem...,Langstreckenfahrzeug daher die hohe Kilometerl...,4 (Green),7,nan
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,85 kW,UsedDiesel (Particulate Filter),1,nan,nan,99 g CO2/km (comb),nan,nan,Black,Metallic,mythosschwarz metallic,"Cloth, Black",Sedans,4,4,nan,Automatic,"1,598 cc",nan,nan,front,Diesel (Particulate Filter),['3.8 l/100 km (comb)']['4.4 l/100 km (city)']...,99 g CO2/km (comb),Euro 6,Air conditioningCruise controlElectrical side ...,MP3On-board computer,Alloy wheelsVoice Control,ABSCentral door lockDaytime running lightsDriv...,Fahrzeug-Nummer: AM-95365 Ehem. UPE 24.640 EUR...,4 (Green),nan,nan
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,66 kW,UsedDiesel (Particulate Filter),1,nan,nan,nan,99 g CO2/km (comb),Euro 6,Brown,Metallic,nan,nan,Sedans,3,4,nan,Automatic,"1,422 cc",3,"1,195 kg",nan,Diesel (Particulate Filter),['3.8 l/100 km (comb)']['4.3 l/100 km (city)']...,99 g CO2/km (comb),Euro 6,Air suspensionArmrestAuxiliary heatingElectric...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport seatsVoice Control,ABSAlarm systemCentral door lock with remote c...,Audi A1: - 1e eigenaar - Perfecte staat: schad...,nan,6,nan
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,None,"16,200 km",05/2016,1 previous owner,66 kW,UsedDiesel (Particulate Filter),1,nan,Yes109 g CO2/km (comb),Euro 6,4 (Green),,Black,Metallic,Mythosschwarz Metallic,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,"1,422 cc",3,nan,front,Diesel (Particulate Filter),['4.1 l/100 km (comb)']['4.6 l/100 km (city)']...,109 g CO2/km (comb),Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport packageSport suspensionVoice...,ABSCentral door lockDriver-side airbagElectron...,Technik & Sicherheit:Xenon plusKlimaautomatikE...,nan,nan,Germany


In [281]:
missing_values = [" ", "Nan", None, "None", "NaN", "", "nan", "[][][]"]

In [282]:
missing = sc.isin(missing_values)

In [283]:
missing.sum()

url                          0
make_model                   0
short_description           46
body_type                   60
price                        0
vat                       4513
km                           0
registration                 0
hp                           0
Type                         0
Previous_Owners           6640
Next_Inspection          12384
Inspection_new           11987
Warranty                  6763
Full_Service              9689
Non_smoking_Vehicle      12459
Body_Color                 597
Paint_Type                5772
Body_Color_Original       3759
Upholstery                3720
Body                        60
Nr._of_Doors               212
Nr._of_Seats               977
Model_Code               10941
Gearing_Type                 0
Displacement               496
Cylinders                 5680
Weight                    6974
Drive_chain               6858
Fuel                         0
Consumption                  0
CO2_Emission              2436
Emission

In [284]:
sc = sc.mask(missing, np.NaN)

In [285]:
sc.isnull().sum()

url                          0
make_model                   0
short_description           46
body_type                   60
price                        0
vat                       4513
km                           0
registration                 0
hp                           0
Type                         2
Previous_Owners           6640
Next_Inspection          12384
Inspection_new           11987
Warranty                  6763
Full_Service              9689
Non_smoking_Vehicle      12459
Body_Color                 597
Paint_Type                5772
Body_Color_Original       3759
Upholstery                3720
Body                        60
Nr._of_Doors               212
Nr._of_Seats               977
Model_Code               10941
Gearing_Type                 0
Displacement               496
Cylinders                 5680
Weight                    6974
Drive_chain               6858
Fuel                         0
Consumption               1906
CO2_Emission              2436
Emission

In [15]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,66 kW,UsedDiesel (Particulate Filter),\n2\n,\n06/2021\n\n99 g CO2/km (comb)\n,\nYes\n\nEuro 6\n,\n\n\n4 (Green)\n,\n\n,\n\n,\nBlack\n,\nMetallic\n,\nMythosschwarz\n,"\nCloth, Black\n",\nSedans\n,\n5\n,\n5\n,\n0588/BDF\n,\nAutomatic\n,"\n1,422 cc\n",\n3\n,"\n1,220 kg\n",\nfront\n,\nDiesel (Particulate Filter)\n,['3.8 l/100 km (comb)']['4.3 l/100 km (city)']...,\n99 g CO2/km (comb)\n,\nEuro 6\n,Air conditioningArmrestAutomatic climate contr...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsCatalytic ConverterVoice Control,ABSCentral door lockDaytime running lightsDriv...,\nSicherheit: Deaktivierung für Beifahrer-Airb...,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,141 kW,UsedGasoline,NaN,NaN,NaN,NaN,NaN,NaN,\nRed\n,NaN,NaN,"\nCloth, Grey\n",\nSedans\n,\n3\n,\n4\n,\n0588/BCY\n,\nAutomatic\n,"\n1,798 cc\n",\n4\n,"\n1,255 kg\n",\nfront\n,\nGasoline\n,['5.6 l/100 km (comb)']['7.1 l/100 km (city)']...,\n129 g CO2/km (comb)\n,\nEuro 6\n,Air conditioningAutomatic climate controlHill ...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsSport seatsSport suspensionVoice C...,ABSCentral door lockCentral door lock with rem...,\nLangstreckenfahrzeug daher die hohe Kilomete...,\n4 (Green)\n,\n7\n,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,85 kW,UsedDiesel (Particulate Filter),\n1\n,NaN,NaN,\n\n\n99 g CO2/km (comb)\n,NaN,NaN,\nBlack\n,\nMetallic\n,\nmythosschwarz metallic\n,"\nCloth, Black\n",\nSedans\n,\n4\n,\n4\n,NaN,\nAutomatic\n,"\n1,598 cc\n",NaN,NaN,\nfront\n,\nDiesel (Particulate Filter)\n,['3.8 l/100 km (comb)']['4.4 l/100 km (city)']...,\n99 g CO2/km (comb)\n,\nEuro 6\n,Air conditioningCruise controlElectrical side ...,MP3On-board computer,Alloy wheelsVoice Control,ABSCentral door lockDaytime running lightsDriv...,\nFahrzeug-Nummer: AM-95365 Ehem. UPE 24.640 E...,\n4 (Green)\n,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,66 kW,UsedDiesel (Particulate Filter),\n1\n,NaN,NaN,NaN,\n\n\n99 g CO2/km (comb)\n,\n\n\nEuro 6\n,\nBrown\n,\nMetallic\n,NaN,NaN,\nSedans\n,\n3\n,\n4\n,NaN,\nAutomatic\n,"\n1,422 cc\n",\n3\n,"\n1,195 kg\n",NaN,\nDiesel (Particulate Filter)\n,['3.8 l/100 km (comb)']['4.3 l/100 km (city)']...,\n99 g CO2/km (comb)\n,\nEuro 6\n,Air suspensionArmrestAuxiliary heatingElectric...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport seatsVoice Control,ABSAlarm systemCentral door lock with remote c...,\nAudi A1: - 1e eigenaar - Perfecte staat: sch...,NaN,\n6\n,NaN
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,None,"16,200 km",05/2016,1 previous owner,66 kW,UsedDiesel (Particulate Filter),\n1\n,NaN,\nYes\n\n109 g CO2/km (comb)\n,\n\n\nEuro 6\n,\n\n\n4 (Green)\n,\n\n,\nBlack\n,\nMetallic\n,\nMythosschwarz Metallic\n,"\nCloth, Black\n",\nSedans\n,\n5\n,\n5\n,\n0588/BDF\n,\nAutomatic\n,"\n1,422 cc\n",\n3\n,NaN,\nfront\n,\nDiesel (Particulate Filter)\n,['4.1 l/100 km (comb)']['4.6 l/100 km (city)']...,\n109 g CO2/km (comb)\n,\nEuro 6\n,A

In [22]:
cl_parantez

['Type',
 'Previous_Owners',
 'Next_Inspection',
 'Inspection_new',
 'Warranty',
 'Full_Service',
 'Non_smoking_Vehicle',
 'Body_Color',
 'Paint_Type',
 'Body_Color_Original',
 'Upholstery',
 'Body',
 'Nr._of_Doors',
 'Nr._of_Seats',
 'Model_Code',
 'Gearing_Type',
 'Displacement',
 'Cylinders',
 'Weight',
 'Drive_chain',
 'Fuel',
 'Consumption',
 'CO2_Emission',
 'Emission_Class',
 'Comfort_&_Convenience',
 'Entertainment_&_Media',
 'Extras',
 'Safety_&_Security',
 'description',
 'Emission_Label',
 'Gears',
 'Country_version']

In [190]:
cl

['url',
 'make_model',
 'short_description',
 'body_type',
 'price',
 'vat',
 'km',
 'registration',
 'prev_owner',
 'hp',
 'Type',
 'Previous_Owners',
 'Next_Inspection',
 'Inspection_new',
 'Warranty',
 'Full_Service',
 'Non_smoking_Vehicle',
 'Body_Color',
 'Paint_Type',
 'Body_Color_Original',
 'Upholstery',
 'Body',
 'Nr._of_Doors',
 'Nr._of_Seats',
 'Model_Code',
 'Gearing_Type',
 'Displacement',
 'Cylinders',
 'Weight',
 'Drive_chain',
 'Fuel',
 'Consumption',
 'CO2_Emission',
 'Emission_Class',
 'Comfort_&_Convenience',
 'Entertainment_&_Media',
 'Extras',
 'Safety_&_Security',
 'description',
 'Emission_Label',
 'Gears',
 'Country_version']

In [257]:
cl_numeric = ["km", "hp", "Displacement", "Weight", "Consumption", "CO2_Emission", "Emission_Label"]
cl_numeric

['km',
 'hp',
 'Displacement',
 'Weight',
 'Consumption',
 'CO2_Emission',
 'Emission_Label']

In [286]:
for i in cl_numeric:
    sc[i] = sc[i].str.replace("\D", "")

In [224]:
sc[["Displacement","Cylinders"]]

,Displacement,Cylinders
0,1422,3
1,1798,4
2,1598,NaN
3,1422,3
4,1422,3
...,...,...
15914,1997,4
15915,1798,4
15916,1997,4
15917,1997,4


In [289]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version,Next_Inspection_1
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,56013,01/2016,66,UsedDiesel (Particulate Filter),2,06/202199 g CO2/km (comb),YesEuro 6,4 (Green),NaN,NaN,Black,Metallic,Mythosschwarz,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,1422,3,1220,front,Diesel (Particulate Filter),381004310035100,992,Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsCatalytic ConverterVoice Control,ABSCentral door lockDaytime running lightsDriv...,Sicherheit: Deaktivierung für Beifahrer-Airbag...,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,80000,03/2017,141,UsedGasoline,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,"Cloth, Grey",Sedans,3,4,0588/BCY,Automatic,1798,4,1255,front,Gasoline,561007110047100,1292,Euro 6,Air conditioningAutomatic climate controlHill ...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsSport seatsSport suspensionVoice C...,ABSCentral door lockCentral door lock with rem...,Langstreckenfahrzeug daher die hohe Kilometerl...,4,7,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,83450,02/2016,85,UsedDiesel (Particulate Filter),1,NaN,NaN,99 g CO2/km (comb),NaN,NaN,Black,Metallic,mythosschwarz metallic,"Cloth, Black",Sedans,4,4,NaN,Automatic,1598,NaN,NaN,front,Diesel (Particulate Filter),381004410034100,992,Euro 6,Air conditioningCruise controlElectrical side ...,MP3On-board computer,Alloy wheelsVoice Control,ABSCentral door lockDaytime running lightsDriv...,Fahrzeug-Nummer: AM-95365 Ehem. UPE 24.640 EUR...,4,NaN,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,NaN,73000,08/2016,66,UsedDiesel (Particulate Filter),1,NaN,NaN,NaN,99 g CO2/km (comb),Euro 6,Brown,Metallic,NaN,NaN,Sedans,3,4,NaN,Automatic,1422,3,1195,NaN,Diesel (Particulate Filter),381004310035100,992,Euro 6,Air suspensionArmrestAuxiliary heatingElectric...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport seatsVoice Control,ABSAlarm systemCentral door lock with remote c...,Audi A1: - 1e eigenaar - Perfecte staat: schad...,NaN,6,NaN,NaN
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,NaN,16200,05/2016,66,UsedDiesel (Particulate Filter),1,NaN,Yes109 g CO2/km (comb),Euro 6,4 (Green),NaN,Black,Metallic,Mythosschwarz Metallic,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,1422,3,NaN,front,Diesel (Particulate Filter),411004610038100,1092,Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport packageSport suspensionVoice...,ABSCentral door lockDriver-side airbagElectron...,Technik & Sicherheit:Xenon plusKlimaautomatikE...,NaN,NaN,Germany,NaN


In [302]:
sc["Next_Inspection"]

0         06/202199 g CO2/km (comb)
1                               NaN
2                               NaN
3                               NaN
4                               NaN
                    ...            
15914                           NaN
15915    01/2022168 g CO2/km (comb)
15916                           NaN
15917                           NaN
15918                           NaN
Name: Next_Inspection, Length: 15919, dtype: object

In [305]:
sc["Next_Inspection"].str.match(r'(\d\d\/\d\d\d\d)')

0        True
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
15914     NaN
15915    True
15916     NaN
15917     NaN
15918     NaN
Name: Next_Inspection, Length: 15919, dtype: object

In [310]:
sc["Next_Inspection"] = sc["Next_Inspection"].str.findall(r'(\d\d\/\d\d\d\d)')

In [312]:
sc["Next_Inspection"]

0        [06/2021]
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
15914          NaN
15915    [01/2022]
15916          NaN
15917          NaN
15918          NaN
Name: Next_Inspection, Length: 15919, dtype: object

In [316]:
sc["Next_Inspection"].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[06/2021]    471
[03/2021]    210
[05/2021]    180
[04/2021]    171
[02/2021]    168
            ... 
[01/1955]      1
[12/2017]      1
[12/2022]      1
[01/1999]      1
[11/2022]      1
Name: Next_Inspection, Length: 77, dtype: int64

In [314]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version,Next_Inspection_1
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,56013,01/2016,66,UsedDiesel (Particulate Filter),2,[06/2021],YesEuro 6,4 (Green),NaN,NaN,Black,Metallic,Mythosschwarz,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,1422,3,1220,front,Diesel (Particulate Filter),381004310035100,992,Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsCatalytic ConverterVoice Control,ABSCentral door lockDaytime running lightsDriv...,Sicherheit: Deaktivierung für Beifahrer-Airbag...,NaN,NaN,NaN,[06/2021]
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,80000,03/2017,141,UsedGasoline,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,"Cloth, Grey",Sedans,3,4,0588/BCY,Automatic,1798,4,1255,front,Gasoline,561007110047100,1292,Euro 6,Air conditioningAutomatic climate controlHill ...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsSport seatsSport suspensionVoice C...,ABSCentral door lockCentral door lock with rem...,Langstreckenfahrzeug daher die hohe Kilometerl...,4,7,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,83450,02/2016,85,UsedDiesel (Particulate Filter),1,NaN,NaN,99 g CO2/km (comb),NaN,NaN,Black,Metallic,mythosschwarz metallic,"Cloth, Black",Sedans,4,4,NaN,Automatic,1598,NaN,NaN,front,Diesel (Particulate Filter),381004410034100,992,Euro 6,Air conditioningCruise controlElectrical side ...,MP3On-board computer,Alloy wheelsVoice Control,ABSCentral door lockDaytime running lightsDriv...,Fahrzeug-Nummer: AM-95365 Ehem. UPE 24.640 EUR...,4,NaN,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,NaN,73000,08/2016,66,UsedDiesel (Particulate Filter),1,NaN,NaN,NaN,99 g CO2/km (comb),Euro 6,Brown,Metallic,NaN,NaN,Sedans,3,4,NaN,Automatic,1422,3,1195,NaN,Diesel (Particulate Filter),381004310035100,992,Euro 6,Air suspensionArmrestAuxiliary heatingElectric...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport seatsVoice Control,ABSAlarm systemCentral door lock with remote c...,Audi A1: - 1e eigenaar - Perfecte staat: schad...,NaN,6,NaN,NaN
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,NaN,16200,05/2016,66,UsedDiesel (Particulate Filter),1,NaN,Yes109 g CO2/km (comb),Euro 6,4 (Green),NaN,Black,Metallic,Mythosschwarz Metallic,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,1422,3,NaN,front,Diesel (Particulate Filter),411004610038100,1092,Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport packageSport suspensionVoice...,ABSCentral door lockDriver-side airbagElectron...,Technik & Sicherheit:Xenon plusKlimaautomatikE...,NaN,NaN,Germany,NaN


In [320]:
sc.dtypes

url                      object
make_model               object
short_description        object
body_type                object
price                     int64
vat                      object
km                       object
registration             object
hp                       object
Type                     object
Previous_Owners          object
Next_Inspection          object
Inspection_new           object
Warranty                 object
Full_Service             object
Non_smoking_Vehicle      object
Body_Color               object
Paint_Type               object
Body_Color_Original      object
Upholstery               object
Body                     object
Nr._of_Doors             object
Nr._of_Seats             object
Model_Code               object
Gearing_Type             object
Displacement             object
Cylinders                object
Weight                   object
Drive_chain              object
Fuel                     object
Consumption              object
CO2_Emis

In [317]:
sc["body_type"].value_counts()

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [ ]:
#### Filling with '0'

In [57]:
sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]] = sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]].fillna(0)

In [ ]:
#### Removing '\n' 

In [58]:
sc[[ "Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]] = sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]].replace("\n","", regex = True)

In [ ]:
#### Removing ',' 

In [59]:
sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]] = sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]].replace(",","", regex = True)

In [60]:
sc.isnull().sum()

url                         0
make_model                  0
short_description          46
body_type                  60
price                       0
vat                      4513
km                          0
registration                0
prev_owner               6828
hp                          0
Type                        2
Previous_Owners             0
Next_Inspection             0
Inspection_new              0
Warranty                    0
Full_Service                0
Non_smoking_Vehicle         0
Body_Color                  0
Paint_Type                  0
Body_Color_Original         0
Upholstery                  0
Body                        0
Nr._of_Doors                0
Nr._of_Seats                0
Model_Code                  0
Gearing_Type                0
Displacement                0
Cylinders                   0
Weight                      0
Drive_chain                 0
Fuel                        0
Consumption              1906
CO2_Emission                0
Emission_C

In [129]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,66 kW,UsedDiesel (Particulate Filter),2,06/202199 g CO2/km (comb),YesEuro 6,4 (Green),,,Black,Metallic,Mythosschwarz,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,"1,422 cc",3,"1,220 kg",front,Diesel (Particulate Filter),['3.8 l/100 km (comb)']['4.3 l/100 km (city)']...,99 g CO2/km (comb),Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsCatalytic ConverterVoice Control,ABSCentral door lockDaytime running lightsDriv...,Sicherheit: Deaktivierung für Beifahrer-Airbag...,nan,nan,nan
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,141 kW,UsedGasoline,nan,nan,nan,nan,nan,nan,Red,nan,nan,"Cloth, Grey",Sedans,3,4,0588/BCY,Automatic,"1,798 cc",4,"1,255 kg",front,Gasoline,['5.6 l/100 km (comb)']['7.1 l/100 km (city)']...,129 g CO2/km (comb),Euro 6,Air conditioningAutomatic climate controlHill ...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsSport seatsSport suspensionVoice C...,ABSCentral door lockCentral door lock with rem...,Langstreckenfahrzeug daher die hohe Kilometerl...,4 (Green),7,nan
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,85 kW,UsedDiesel (Particulate Filter),1,nan,nan,99 g CO2/km (comb),nan,nan,Black,Metallic,mythosschwarz metallic,"Cloth, Black",Sedans,4,4,nan,Automatic,"1,598 cc",nan,nan,front,Diesel (Particulate Filter),['3.8 l/100 km (comb)']['4.4 l/100 km (city)']...,99 g CO2/km (comb),Euro 6,Air conditioningCruise controlElectrical side ...,MP3On-board computer,Alloy wheelsVoice Control,ABSCentral door lockDaytime running lightsDriv...,Fahrzeug-Nummer: AM-95365 Ehem. UPE 24.640 EUR...,4 (Green),nan,nan
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,66 kW,UsedDiesel (Particulate Filter),1,nan,nan,nan,99 g CO2/km (comb),Euro 6,Brown,Metallic,nan,nan,Sedans,3,4,nan,Automatic,"1,422 cc",3,"1,195 kg",nan,Diesel (Particulate Filter),['3.8 l/100 km (comb)']['4.3 l/100 km (city)']...,99 g CO2/km (comb),Euro 6,Air suspensionArmrestAuxiliary heatingElectric...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport seatsVoice Control,ABSAlarm systemCentral door lock with remote c...,Audi A1: - 1e eigenaar - Perfecte staat: schad...,nan,6,nan
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,None,"16,200 km",05/2016,1 previous owner,66 kW,UsedDiesel (Particulate Filter),1,nan,Yes109 g CO2/km (comb),Euro 6,4 (Green),,Black,Metallic,Mythosschwarz Metallic,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,"1,422 cc",3,nan,front,Diesel (Particulate Filter),['4.1 l/100 km (comb)']['4.6 l/100 km (city)']...,109 g CO2/km (comb),Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport packageSport suspensionVoice...,ABSCentral door lockDriver-side airbagElectron...,Technik & Sicherheit:Xenon plusKlimaautomatikE...,nan,nan,Germany


In [132]:
sc["vat"].value_counts()

VAT deductible      10980
Price negotiable      426
Name: vat, dtype: int64

In [136]:
sc["vat"].value_counts()

VAT deductible      10980
Price negotiable      426
Name: vat, dtype: int64

In [ ]:
sc["vat"].fillna("VAT deductible")

In [135]:
sc["make_model"].value_counts()

Audi A3           3097
Audi A1           2614
Opel Insignia     2598
Opel Astra        2526
Opel Corsa        2219
Renault Clio      1839
Renault Espace     991
Renault Duster      34
Audi A2              1
Name: make_model, dtype: int64

In [105]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,66 kW,UsedDiesel (Particulate Filter),2,06/202199 g CO2/km (comb),YesEuro 6,4 (Green),NaN,NaN,Black,Metallic,Mythosschwarz,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,"1,422 cc",3,"1,220 kg",front,Diesel (Particulate Filter),['3.8 l/100 km (comb)']['4.3 l/100 km (city)']...,99 g CO2/km (comb),Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsCatalytic ConverterVoice Control,ABSCentral door lockDaytime running lightsDriv...,Sicherheit: Deaktivierung für Beifahrer-Airbag...,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,NaN,141 kW,UsedGasoline,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,"Cloth, Grey",Sedans,3,4,0588/BCY,Automatic,"1,798 cc",4,"1,255 kg",front,Gasoline,['5.6 l/100 km (comb)']['7.1 l/100 km (city)']...,129 g CO2/km (comb),Euro 6,Air conditioningAutomatic climate controlHill ...,BluetoothHands-free equipmentOn-board computer...,Alloy wheelsSport seatsSport suspensionVoice C...,ABSCentral door lockCentral door lock with rem...,Langstreckenfahrzeug daher die hohe Kilometerl...,4 (Green),7,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,85 kW,UsedDiesel (Particulate Filter),1,NaN,NaN,99 g CO2/km (comb),NaN,NaN,Black,Metallic,mythosschwarz metallic,"Cloth, Black",Sedans,4,4,NaN,Automatic,"1,598 cc",NaN,NaN,front,Diesel (Particulate Filter),['3.8 l/100 km (comb)']['4.4 l/100 km (city)']...,99 g CO2/km (comb),Euro 6,Air conditioningCruise controlElectrical side ...,MP3On-board computer,Alloy wheelsVoice Control,ABSCentral door lockDaytime running lightsDriv...,Fahrzeug-Nummer: AM-95365 Ehem. UPE 24.640 EUR...,4 (Green),NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,NaN,"73,000 km",08/2016,1 previous owner,66 kW,UsedDiesel (Particulate Filter),1,NaN,NaN,NaN,99 g CO2/km (comb),Euro 6,Brown,Metallic,NaN,NaN,Sedans,3,4,NaN,Automatic,"1,422 cc",3,"1,195 kg",NaN,Diesel (Particulate Filter),['3.8 l/100 km (comb)']['4.3 l/100 km (city)']...,99 g CO2/km (comb),Euro 6,Air suspensionArmrestAuxiliary heatingElectric...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport seatsVoice Control,ABSAlarm systemCentral door lock with remote c...,Audi A1: - 1e eigenaar - Perfecte staat: schad...,NaN,6,NaN
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,NaN,"16,200 km",05/2016,1 previous owner,66 kW,UsedDiesel (Particulate Filter),1,NaN,Yes109 g CO2/km (comb),Euro 6,4 (Green),NaN,Black,Metallic,Mythosschwarz Metallic,"Cloth, Black",Sedans,5,5,0588/BDF,Automatic,"1,422 cc",3,NaN,front,Diesel (Particulate Filter),['4.1 l/100 km (comb)']['4.6 l/100 km (city)']...,109 g CO2/km (comb),Euro 6,Air conditioningArmrestAutomatic climate contr...,BluetoothCD playerHands-free equipmentMP3On-bo...,Alloy wheelsSport packageSport suspensionVoice...,ABSCentral door lockDriver-side airbagElectron...,Technik & Sicherheit:Xenon plusKlimaautomatikE...,NaN,NaN,Germany


In [152]:
sc.shape

(15919, 42)

In [ ]:
# pd.get_dummies(df, prefix=['col1', 'col2'])


In [154]:
sc.dtypes

url                      object
make_model               object
short_description        object
body_type                object
price                     int64
vat                      object
km                       object
registration             object
prev_owner               object
hp                       object
Type                     object
Previous_Owners          object
Next_Inspection          object
Inspection_new           object
Warranty                 object
Full_Service             object
Non_smoking_Vehicle      object
Body_Color               object
Paint_Type               object
Body_Color_Original      object
Upholstery               object
Body                     object
Nr._of_Doors             object
Nr._of_Seats             object
Model_Code               object
Gearing_Type             object
Displacement             object
Cylinders                object
Weight                   object
Drive_chain              object
Fuel                     object
Consumpt

In [ ]:
# replace np.NaN

In [ ]:
#sc[sc.isnull().any(axis=1)]

In [ ]:
# business.categories=business.categories.apply(lambda row: re.split(", | & ",row))

In [58]:
# jeopardy_data["Float_Value"]=jeopardy_data["Value"].apply(lambda x:0.0 if pd.isna(x) else x.replace('$',''))
